In [4]:
def makevoicegraph(filepath,savename,date,stt):
    
    import warnings
    warnings.filterwarnings(action='ignore') 
    from keras.models import load_model
    from python_speech_features import mfcc
    import matplotlib.pyplot as plt
    from matplotlib import font_manager, rc
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
    %matplotlib inline
    import numpy as np
    import scipy.io.wavfile as wav
    import moviepy.editor as mp
    import seaborn as sns
    import pandas as pd
    
    
    (rate,sig) = wav.read(filepath)
    mfcc_feat = mfcc(sig,rate)
    data = mfcc_feat[:len(mfcc_feat)//(200)*200].reshape(1,-1).reshape(len(mfcc_feat)//200,200,13)
    model = load_model('model/voicelstm-0.9541.hdf5')
    pre = model.predict(data)
    blue, = sns.color_palette("muted", 1)
    x = np.arange(0,len(pre))
    y = 1-pre.reshape(1,-1)[0]
    fig, ax = plt.subplots(figsize=(30,8))
    plt.title('목소리 어색함 분석 차트',fontdict = {'fontsize' : 50})
    
    stt = pd.DataFrame(stt,columns=['word','start','end'])
    df = pd.DataFrame(range(1,len(x)+1),columns=['time'])
    df['str'] = ''
    for i in range(0,len(stt)):
        df['str'][df['time']==stt.loc[i,'start'].astype(int)] = df['str'][df['time']==stt.loc[i,'start'].astype(int)] + stt.loc[i,'word'] 
    
    ax.plot(x, y, color=blue, lw=1.5)
    ax.fill_between(x, 0, y, alpha=.3)
    ax.set(xlim=(0,None), ylim=(0, None))
    s = np.pi/3
    for i in range(0,len(df)):
        s = s+np.pi/3
        yp = y[i]-0.03+np.sin(s)*0.03
        if (yp<0):
            yp = 0.01
        plt.text(x[i]-0.5,yp ,df['str'][i],fontdict={'fontsize':12}) 
    plt.xlabel('값이 1에 가까울수록 어색함', fontdict={'fontsize':20})
    plt.savefig('result/'+savename+date+'/'+savename+date+'voice.png')
    print('voicegraph saved in '+'result/'+savename+date+'/'+savename+date+'voice.png')